## Imports

In [8]:
import sys  
!{sys.executable} -m pip install --user timezonefinder

  Using cached timezonefinder-5.2.0-py36.py37.py38-none-any.whl (43.0 MB)


In [9]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import datetime
import os
from timezonefinder import TimezoneFinder
from pytz import timezone
import pytz
import glob
import pickle
import math
import itertools

tf = TimezoneFinder(in_memory=True)

utc = pytz.utc

## Function declarations

In [11]:
"""
Example script that scrapes data from the IEM ASOS download service
"""
# Python 2 and 3: alternative 4
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"

output_dir = os.path.relpath('US_METAR_data')
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    
def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode('utf-8')
            if data is not None and not data.startswith('ERROR'):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""


def get_stations_from_filelist(filename):
    """Build a listing of stations from a simple file listing the stations.
    The file should simply have one station per line.
    """
    stations = []
    for line in open(filename):
        stations.append(line.strip())
    return stations


def get_stations_from_networks():
    """Build a station list by using a bunch of IEM networks."""
    stations = []
    US_states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
     MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
     WA WI WV WY"""
    # IEM quirk to have Iowa AWOS sites in its own labeled network
    networks = ['AWOS']
    for state in US_states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = ("https://mesonet.agron.iastate.edu/"
               "geojson/network/%s.geojson") % (network,)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict['features']:
            stations.append(site['properties']['sid'])
    return stations


def offset(target):
    """
    returns a location's time zone offset from UTC.
    """
    today = datetime.datetime.now()
    tz_target = timezone(tf.certain_timezone_at(lat=target['lat'], lng=target['lng']))
    # ATTENTION: tz_target could be None! handle error case

    # today_target = tz_target.localize(today)
    # today_utc = utc.localize(today)
    # offset = today_utc - today_target
    offset = tz_target.utcoffset(today)

    # if `today` is in summer time while the target isn't, you may want to substract the DST
    offset -= tz_target.dst(today)
    return offset

### Use this to only pull in more data when you need

In [12]:
"""Our main method"""
# timestamps in UTC to request data for
startts = datetime.datetime(2015, 1, 1)
# endts = datetime.datetime(2018, 12, 31)
endts = datetime.datetime.now().date()

service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

service += startts.strftime('year1=%Y&month1=%m&day1=%d&')
service += endts.strftime('year2=%Y&month2=%m&day2=%d&')

# Two examples of how to specify a list of stations
# stations = get_stations_from_networks()
# stations = get_stations_from_filelist("mystations.txt")

"""
KBHM = Birmingham, Alabama
PANC = Anchorage, Alaska
KPHX = Pheonix, Arizona
KLIT = Little Rock, Arkansas
KLAX = Los Angeles, California
KDEN = Denver, Colorado
KBDL = Hartford, Connecticut
KILG = Wilmington, Delaware
KMIA = Miami, Florida
KATL = Atlanta, Georgia
PHNL = Honolulu, Hawaii
KBOI = Boise, Idaho
KORD = Chicago, Illinois
KIND = Indianapolis, Indiana
KDSM = Des Moines, Iowa
KICT = Wichita, Kansas
KCVG = Cincinnati, Kentucky
KMSY = New Orleans, Louisiana
KPWM = Portland, Maine
KBWI = Baltimore, Maryland
KBOS = Boston, Massacheusetts
KDTW = Detroit, Michigan
KMSP = Minneapolis, Minnesota
KJAN = Jackson, Mississippi
KSTL = St. Louis, Missouri
KBZN = Bozeman, Montana
KOMA = Omaha, Nebraska
KLAS = Las Vegas, Nevada
KMHT = New Hampshire
KEWR = Newark, New Jersey
KABQ = Albuquerque, New Mexico
KJFK = New York, New York
KCLT = Charlotte, North Carolina
KFAR = Fargo, North Dakota
KCLE = Cleveland, Ohio
KOKC = Oklahoma City, Oklahoma
KPDX = Portland, Oregon
KPHL = Philadelphia, Pennsylvania
KPVD = Providence, Rhode Island
KCHS = Charleston, South Carolina
KFSD = Sioux Falls, South Dakota
KBNA = Nashville, Tennessee
KDFW = Dallas-Fort Worth, Texas
KSLC = Salt Lake City, Utah
KBTV = Burlington, Vermont
KDCA = Washington D.C, Virginia
KSEA = Seattle, Washington
KCRW = Charleston, West Virginia
KMKE = Milwaukee, Wisconsin
KJAC = Jackson, Wyoming
"""

stations = ['KBHM','PANC','KPHX','KLIT','KLAX','KDEN','KBDL','KILG','KMIA','KATL','PHNL','KBOI','KORD', 'KIND',
            'KDSM', 'KICT', 'KCVG', 'KMSY', 'KPWM', 'KBWI', 'KBOS', 'KDTW', 'KMSP', 'KJAN', 'KSTL', 'KBZN', 
            'KOMA', 'KLAS', 'KMHT', 'KEWR', 'KABQ', 'KJFK', 'KCLT', 'KFAR', 'KCLE', 'KOKC', 'KPDX', 'KPHL', 
            'KPVD', 'KCHS', 'KFSD', 'KBNA', 'KDFW', 'KSLC', 'KBTV', 'KDCA', 'KSEA', 'KCRW', 'KMKE', 'KJAC']

for station in stations:
    uri = '%s&station=%s' % (service, station)
    print('Downloading: %s' % (station, ))
    data = download_data(uri)
    outfn = os.path.join(output_dir, '%s_%s_%s.txt' % (station, startts.strftime("%Y%m%d%H%M"),
                              endts.strftime("%Y%m%d%H%M")))
    out = open(outfn, 'w')
    out.write(data)
    out.close()

Downloading: KBHM
Downloading: PANC
Downloading: KPHX
Downloading: KLIT
Downloading: KLAX
Downloading: KDEN
Downloading: KBDL
Downloading: KILG
Downloading: KMIA
Downloading: KATL
Downloading: PHNL
Downloading: KBOI
Downloading: KORD
Downloading: KIND
Downloading: KDSM
Downloading: KICT
Downloading: KCVG
Downloading: KMSY
Downloading: KPWM
Downloading: KBWI
Downloading: KBOS
Downloading: KDTW
Downloading: KMSP
Downloading: KJAN
Downloading: KSTL
Downloading: KBZN
Downloading: KOMA
Downloading: KLAS
Downloading: KMHT
Downloading: KEWR
Downloading: KABQ
Downloading: KJFK
Downloading: KCLT
Downloading: KFAR
Downloading: KCLE
Downloading: KOKC
Downloading: KPDX
Downloading: KPHL
Downloading: KPVD
Downloading: KCHS
Downloading: KFSD
Downloading: KBNA
Downloading: KDFW
Downloading: KSLC
Downloading: KBTV
Downloading: KDCA
Downloading: KSEA
Downloading: KCRW
Downloading: KMKE
Downloading: KJAC


## Processing
- Converting Sky Condition categories into quantifiable numbers
- Converting datetime string to datetime format
- Adding UTC offset based on location, and **filtering data between 8am - 5pm**
- Getting rid of cloud values that are above 20k feet. Because you can just go under them

In [13]:
# df_yssy = pd.read_csv('output/YSSY_201501010000_201909020000.txt', skiprows=5)
# df = df_yssy[['valid','station','lat','lon','skyc1','skyc2','skyc3', 'skyc4','skyl1','skyl2','skyl3', 'skyl4','metar']]
# df.head()

In [14]:
df = pd.read_csv('US_METAR_data/KLAX_201501010000_202107240000.txt', skiprows=5)
df = df[['valid','station','lat','lon','skyc1','skyc2','skyc3', 'skyc4','skyl1','skyl2','skyl3', 'skyl4','metar']]
df.head(5)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,valid,station,lat,lon,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,metar
0,2015-01-01 00:53,LAX,33.9382,-118.3865,FEW,M,M,M,5000.00,M,M,M,KLAX 010053Z 32010KT 10SM FEW050 12/M11 A3001 ...
1,2015-01-01 01:53,LAX,33.9382,-118.3865,CLR,M,M,M,M,M,M,M,KLAX 010153Z 33006G14KT 10SM CLR 11/M09 A3003 ...
2,2015-01-01 02:53,LAX,33.9382,-118.3865,CLR,M,M,M,M,M,M,M,KLAX 010253Z 34007KT 10SM CLR 10/M09 A3004 RMK...
3,2015-01-01 03:53,LAX,33.9382,-118.3865,CLR,M,M,M,M,M,M,M,KLAX 010353Z 35007KT 10SM CLR 10/M10 A3005 RMK...
4,2015-01-01 04:53,LAX,33.9382,-118.3865,CLR,M,M,M,M,M,M,M,KLAX 010453Z 00000KT 10SM CLR 09/M10 A3006 RMK...


In [15]:
filenames = glob.glob('US_METAR_data/*201501010000_202107240000.txt')
df_list = [pd.read_csv(filename, skiprows=5) for filename in filenames]

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [16]:
concat_df = pd.DataFrame()

for df in df_list:
    # Some clean up
    df = df[['valid','station','lat','lon','skyc1','skyc2','skyc3', 'skyc4','skyl1','skyl2','skyl3', 'skyl4','metar']]
    if df.empty:
        raise Exception('Empty Dataframe')
    df = df.replace('M', np.NaN)
    df.dropna(subset=['metar'], inplace=True) # Omits rows without METAR information
    # if skyc1 is 'CLR' ensure that skyl1 is 12k ft. (find reference for this)
    df.loc[df.skyc1 == 'CLR', 'skyl1'] = 12000

    # Converting Sky Condition categories into quantifiable numbers
    weather_dict = {'FEW' : 1.5,  # Few
                    'SCT' : 3.5,  # Scattered
                    'BKN' : 6,    # Broken
                    'OVC' : 8,    # Overcast
                    'SKC' : 0,    # Sky Clear
                    'NCD' : 0,    # No Cloud Detected
                    'NSC' : 0,    # No Significant Clouds
                    'CLR' : 0,    # Clear: seems like a US term
                    'CAVOK' : 0}  # Celing and Visibility OK
                   #'M' : np.NaN}
    df['skyc1'] = df['skyc1'].map(weather_dict)
    df['skyc2'] = df['skyc2'].map(weather_dict)
    df['skyc3'] = df['skyc3'].map(weather_dict)
    df['skyc4'] = df['skyc4'].map(weather_dict)
    df = df.astype({'skyl1': 'float32', 'skyl2': 'float32', 'skyl3': 'float32', 'skyl4': 'float32'})
   

    # Adding UTC offset based on location, and filtering data between 8am - 5pm
    df.valid = pd.to_datetime(df.valid, infer_datetime_format=True)
    location = dict({'lat':df['lat'][0], 'lng':df['lon'][0]})
    timedelta = offset(location)
    df.valid = df.valid + timedelta
    df = df[df.valid.dt.strftime('%H:%M:%S').between('08:00:00','17:00:00')]

    for index, row in df.iterrows():
        # If 'CAVOK' is the Metar, report 0 okta at 5k ft and 0.5 Okta up to 10ft
        # This is a very rough heuristic, research on METAR more before solidifying this
        if 'CAVOK' in df.at[index,'metar'].split(' '):
            if row.isnull()['skyc1']:
                df.at[index,'skyc1'] = 0
                df.at[index,'skyl1'] = 5000
            if row.isnull()['skyc2']:
                df.at[index,'skyc2'] = 0.5
                df.at[index,'skyl2'] = 10000
        # Omitting readings above 20k ft
        if row['skyl3'] > 20000:
            df.at[index,'skyc3'] = np.NaN
            df.at[index,'skyl3'] = np.NaN
        if row['skyl4'] > 20000:
            df.at[index,'skyc4'] = np.NaN
            df.at[index,'skyl4'] = np.NaN

    # Averaging cloud cover information for all the categories - dumb method
    # NOTE: its better to create 'avg_cloud' here instead of after the groupby because
    # all the NaN entries are averaged better. The averages don't match up in the final df because
    # different sky categories have different amounts of NaNs
    df['avg_cloud'] = df[['skyc1', 'skyc2','skyc3','skyc4']].mean(axis=1)
    # if avg_cloud for a location == NaN, use the previous date's value at the same location
    for i in range(1, len(df)):
        if i == 0: continue
        if df.iloc[i].isnull()[13]:
            df.iat[i,13] = df.iat[i-1,13]
    
    concat_df = pd.concat([concat_df, df], sort=False)
    
## Compressing the dataframe - from hours into days of years - Then years into a single year
concat_df = concat_df.groupby([concat_df.valid.dt.strftime("%m/%d"), concat_df.station]).mean()
# concat_df['avg_cloud'] = concat_df[['skyc1', 'skyc2','skyc3','skyc4']].mean(axis=1)

In [17]:
concat_df.head(5)

lat       lon     skyc1     skyc2     skyc3  skyc4  \
valid station                                                           
01/01 ABQ      35.0419 -106.6155  2.568293  5.023207  7.064626    6.0   
      ATL      33.6301  -84.4418  4.205607  6.340000  6.888889    8.0   
      BDL      41.9381  -72.6825  1.939539  5.680851  5.347826    NaN   
      BHM      33.5655  -86.7449  2.685668  6.625616  7.802469    6.0   
      BNA      36.1189  -86.6892  3.841338  6.008696  6.840426    6.0   

                     skyl1        skyl2        skyl3    skyl4  avg_cloud  
valid station                                                             
01/01 ABQ      6443.572266  5275.472656  5916.993164  14000.0   3.319888  
      ATL      3889.860596  6125.756348  8683.901367  13750.0   5.120699  
      BDL      8713.620117  8003.191406  7108.695801      NaN   2.130277  
      BHM      7050.569824  6021.083984  6507.357910  14000.0   3.247213  
      BNA      6071.336426  5639.886719  5055.212891  20000.0   3.858672

### Saving the pickle

In [18]:
file_name = 'output/US_weather_7yr_avg'
# Open the file for writing
with open(file_name,'wb') as my_file_obj:
    pickle.dump(concat_df,my_file_obj)